In [1]:
import re
import pandas as pd
import numpy as np
import PyPDF2

### NaturDoc - TL BL WT 22-23

# Data enrichment:

## Imports:

### Duke:

In [2]:
mod_duke_df = pd.read_csv("../data/DUKE/PIVOT_ETHNOBOT.csv")

### KEW:

##### _Note: reduced and pivoted WCVP datasets:_

As mentioned in the data_transformation notebook, some of the csv files were too large to be uploaded to Github. By creating a copy of wcvp_distribution.csv and wcvp_names.csv containing only columns that were useful to us and by pivoting the entire table, the number of rows and columns was reduced significantly for each file.

In [3]:
dis_df = pd.read_csv("../data/WCVP/STR_REDUCED_PIVOT_wcvp_distribution.csv", sep=",")
names_df = pd.read_csv("../data/WCVP/REDUCED_wcvp_names.csv", sep=",")

### WHO Monographs:

In [4]:
who_01_pdf = open('../data/WHO/monograph_01.pdf', 'rb')

who_01_reader = PyPDF2.PdfReader(who_01_pdf)

who_01_pages = list()

pattern_fi = re.compile("ﬁ")
pattern_fl = re.compile("ﬂ")
pattern_minus = re.compile("–")
pattern_brackets = re.compile("\([\s0-9,-]+\)")
pattern_spacedot = re.compile(" \.")
pattern_spacecomma = re.compile(" ,")
pattern_spacequotesingle = re.compile(" ’")
pattern_spacequotedouble = re.compile(" ”")
pattern_doublespace = re.compile("\s\s+")
pattern_dashspace = re.compile("-\s+")
  
for page in range(len(who_01_reader.pages)):
    page_obj = who_01_reader.pages[page]
    text = page_obj.extract_text()
    text = re.sub(pattern_fi, "fi", text)
    text = re.sub(pattern_fl, "fl", text)
    text = re.sub(pattern_minus, "-", text)
    text = re.sub(pattern_brackets, "", text)
    text = re.sub(pattern_spacedot, ".", text)
    text = re.sub(pattern_spacecomma, ",", text)
    text = re.sub(pattern_spacequotesingle, "’", text)
    text = re.sub(pattern_spacequotedouble, "”", text)
    text = re.sub(pattern_doublespace, " ", text)
    text = re.sub(pattern_dashspace, "", text)
    lines = text.split("\n")
    who_01_pages.append(lines)
    
who_01_pdf.close()

In [5]:
p_01_50 = who_01_pages[7 : 57]
p_51_58 = who_01_pages[75 : 83]
p_59_60 = who_01_pages[57 : 59]
p_61_66 = who_01_pages[69 : 75]
p_67_76 = who_01_pages[59 : 69]
p_77_end = who_01_pages[83 : ]

who_01_pages_corrected = p_01_50 + p_51_58 + p_59_60 + p_61_66 + p_67_76 + p_77_end

## Enriching the Duke dataset:

In [6]:
duke_names = mod_duke_df["TAXON"].unique().tolist()

In [7]:
pattern_formula = re.compile(r"\.[A-Za-z0-9]+.*")

# Definitions with in-keyword:

who_01_def = list()
  
for page in who_01_pages_corrected[3:]:
    max_len = len(page) - 1
    for i, line in enumerate(page):
        if "Definition" in line:
            n_lines = 1
            while page[i + n_lines] != "Synonyms" and i + n_lines < max_len:
                n_lines += 1
            who_01_def.append(" ".join(page[i + 1 : i + n_lines]))

# Vernacular names with in-keyword:

who_01_vernacular = list()
reg_pattern = re.compile(r"\.")
  
for page in who_01_pages_corrected[3:]:
    max_len = len(page) - 1
    for i, line in enumerate(page):
        if "Selected vernacular names" in line:
            n_lines = 1
            while page[i + n_lines] != "Description" and i + n_lines < max_len:
                n_lines += 1
            if i + n_lines == max_len:
                output = " ".join(page[i + 1 : i + n_lines + 1])
            else:
                output = " ".join(page[i + 1 : i + n_lines])
            
            output =  output.replace(".", ",")
            
            # Sometimes taxonomic names appear in the WHO document names as a header of sorts:
            for name in duke_names:
                output = output.replace(name, ",")
            
            output =  output.replace(", ", ",")

            who_01_vernacular.append(output)

# Clinical medicine with in-keyword:

who_01_clinical = list()
  
for page in who_01_pages_corrected[3:]:
    max_len = len(page) - 1
    for i, line in enumerate(page):
        if "Uses supported by clinical data" in line:
            n_lines = 1
            while "Uses described in pharmacopoeias and in traditional" not in page[i + n_lines] and i + n_lines < max_len:
                n_lines += 1
            if i + n_lines == max_len:
                output = " ".join(page[i + 1 : i + n_lines + 1])
                output = re.sub(pattern_formula, ".", output)
            else:
                output = " ".join(page[i + 1 : i + n_lines])
                output = re.sub(pattern_formula, ".", output)

            # in case the description gets cut off, drop the incomplete sentence:
            if "." != output[-1]:
                output = output.split(".")
                output = output[:-1]
                output = ".".join(output)
                output = output + "."
            
            who_01_clinical.append(output)

# Traditional medicine with in-keyword:

who_01_traditional = list()
  
for page in who_01_pages_corrected[3:]:
    max_len = len(page) - 1
    for i, line in enumerate(page):
        if "Uses described in pharmacopoeias and in traditional" in line:
            n_lines = 2
            while "Uses described in folk medicine" not in page[i + n_lines] and i + n_lines < max_len:
                n_lines += 1
            if i + n_lines == max_len:
                output = " ".join(page[i + 2 : i + n_lines + 1])
                output = re.sub(pattern_formula, ".", output)
            else:
                output = " ".join(page[i + 2 : i + n_lines])
                output = re.sub(pattern_formula, ".", output)

            if "." != output[-1]:
                output = output.split(".")
                output = output[:-1]
                output = ".".join(output)
                output = output + "."

            who_01_traditional.append(output)

# Folk with in-keyword:

who_01_folk = list()
  
for page in who_01_pages_corrected[3:]:
    max_len = len(page) - 1
    for i, line in enumerate(page):
        if "Uses described in folk medicine" in line:
            n_lines = 2
            while page[i + n_lines] != "Pharmacology" and i + n_lines < max_len:
                n_lines += 1
            if i + n_lines == max_len:
                output = " ".join(page[i + 2 : i + n_lines + 1])
                output = re.sub(pattern_formula, ".", output)
            else:
                output = " ".join(page[i + 2 : i + n_lines])
                output = re.sub(pattern_formula, ".", output)

            if "." != output[-1]:
                output = output.split(".")
                output = output[:-1]
                output = ".".join(output)
                output = output + "."

            who_01_folk.append(output)

who_01_con = list()
  
for page in who_01_pages_corrected[4:]:
    max_len = len(page) - 1
    for i, line in enumerate(page):
        if "Contraindications" in line and "see" not in line.lower():
            n_lines = 1
            while page[i + n_lines] != "Warnings" and i + n_lines < max_len:
                n_lines += 1
            if i + n_lines == max_len:
                output = " ".join(page[i + 1 : i + n_lines + 1])
                output = re.sub(pattern_formula, ".", output)
            else:
                output = " ".join(page[i + 1 : i + n_lines])
                output = re.sub(pattern_formula, ".", output)

            if "." != output[-1]:
                output = output.split(".")
                output = output[:-1]
                output = ".".join(output)
                output = output + "."

            who_01_con.append(output)
            
who_01_warn = list()
  
for page in who_01_pages_corrected[4:]:
    max_len = len(page) - 1
    for i, line in enumerate(page):
        if "Warnings" in line and "see" not in line.lower():
            n_lines = 1
            while page[i + n_lines] != "Precautions" and i + n_lines < max_len:
                n_lines += 1
            if i + n_lines == max_len:
                output = " ".join(page[i + 1 : i + n_lines + 1])
                output = re.sub(pattern_formula, ".", output)
            else:
                output = " ".join(page[i + 1 : i + n_lines])
                output = re.sub(pattern_formula, ".", output)

            if "." != output[-1]:
                output = output.split(".")
                output = output[:-1]
                output = ".".join(output)
                output = output + "."

            who_01_warn.append(output)
            
who_01_adv = list()

ref_pattern = re.compile(r"[0-9]+\.")  

for page in who_01_pages_corrected[4:]:
    max_len = len(page) - 1
    for i, line in enumerate(page):
        if "Adverse reactions" in line and "see" not in line.lower() and not re.search(ref_pattern, line):
            n_lines = 1
            while page[i + n_lines] != "Posology" and i + n_lines < max_len:
                n_lines += 1
            if i + n_lines == max_len:
                output = " ".join(page[i + 1 : i + n_lines + 1])
                output = re.sub(pattern_formula, ".", output)
            else:
                output = " ".join(page[i + 1 : i + n_lines])
                output = re.sub(pattern_formula, ".", output)

            if "." != output[-1]:
                output = output.split(".")
                output = output[:-1]
                output = ".".join(output)
                output = output + "."

            who_01_adv.append(output)

who_01_pos = list()
  
for page in who_01_pages_corrected[4:]:
    max_len = len(page) - 1
    for i, line in enumerate(page):
        if "Posology" in line and "and" not in line.lower():
            n_lines = 1
            while page[i + n_lines] != "References" and i + n_lines < max_len:
                n_lines += 1
            if i + n_lines == max_len:
                output = " ".join(page[i + 1 : i + n_lines + 1])
                # output = re.sub(pattern_formula, ".", output)
            else:
                output = " ".join(page[i + 1 : i + n_lines])
                # output = re.sub(pattern_formula, ".", output)

            if "." != output[-1]:
                output = output.split(".")
                output = output[:-1]
                output = ".".join(output)
                output = output + "."

            who_01_pos.append(output)

Replace WHO null values with empty string:

In [22]:
null_values = ["No information available.", "None."]

def make_nan(target_list: list):
    for i, value in enumerate(target_list):
        if value in null_values:
            target_list[i] = ""

In [23]:
make_nan(who_01_clinical)
make_nan(who_01_traditional)
make_nan(who_01_folk)
make_nan(who_01_con)
make_nan(who_01_warn)
make_nan(who_01_adv)
make_nan(who_01_pos)

In [25]:
who_01_remedies = list()

for i, definition in enumerate(who_01_def):

    for name in duke_names:

        remedy_dict = dict()
        name = name.split(" ")

        # some taxonomic names have an "x", indicating hybrids
        if "x" in name: 
            name.remove("x")

        if name[0] in definition and name[1] in definition:
            remedy_dict["NAME"] = " ".join(name)
            remedy_dict["DEF"] = definition
            remedy_dict["VERNAC"] = who_01_vernacular[i]
            remedy_dict["CLINICAL"] = who_01_clinical[i]
            remedy_dict["TRADITIONAL"] = who_01_traditional[i] 
            remedy_dict["FOLK"] = who_01_folk[i]
            remedy_dict["CONTRAINDICATION"] = who_01_con[i]
            remedy_dict["WARNING"] = who_01_warn[i]
            remedy_dict["ADVERSE"] = who_01_adv[i]
            remedy_dict["POSOLOGY"] = who_01_pos[i]

        if remedy_dict:
            who_01_remedies.append(remedy_dict)

### Preparing the Duke dataframe:

When reading the modified ETHNOBOT dataset, a new serial index is created, so we need to drop the "Unnamed: 0" column.

In [35]:
mod_duke_df.columns

Index(['Unnamed: 0', 'TAXON', 'ACTIVITY', 'CNAME', 'FAMILY', 'GENUS',
       'SPECIES'],
      dtype='object')

In [27]:
target_cols = ["TAXON", "ACTIVITY", "CNAME", "FAMILY", "GENUS", "SPECIES"]
rich_duke_df = mod_duke_df[target_cols]
rich_duke_df.head()

,TAXON,ACTIVITY,CNAME,FAMILY,GENUS,SPECIES
0,Abelmoschus esculentus,"Abortifacient,Antidote,Boil,Burn,Catarrh,Coffe...","nan,Huang Shu K'Uei,Bamia,Molondron,Quiabeiro,...",Malvaceae,Abelmoschus,esculentus
1,Abelmoschus manihot,"Antitussive,Boil,Cancer,Cancer(Stomach),Catarr...","Tororo-Aoi,nan,Kastuli",Malvaceae,Abelmoschus,manihot
2,Abelmoschus moschatus,"Aphrodisiac,Asthma,Bite(Snake),Boil,Cancer,Car...","Musk Mallow,nan,Moskus,Muskus,Mushk Dana,Amber...",Malvaceae,Abelmoschus,moschatus
3,Abies alba,"Bronchitis,Bruise,Calculus,Catarrh,Cough,Diure...","Abeto,Edeltanne,Beyaz Koknar,Silver Fir,Abeto ...",Pinaceae,Abies,alba
4,Abies balsamea,"Burn,Cancer,Cold,Cough,Heart,Masticatory,Sore,...","nan,Balsam,Canada,Balsam Fir,Fir",Pinaceae,Abies,balsamea


### Adding data from WHO monographs to the reshaped dataset:

Preparing new columns in the enriched dataframe for the information stored in the remedy dictionary. Since we previously replaced all np.NaN values with a string "nan", the default value will be the same here:

In [28]:
rich_duke_df["VERNAC"] = "nan"
rich_duke_df["CLINICAL"] = "nan"
rich_duke_df["TRADITIONAL"] = "nan"
rich_duke_df["FOLK"] = "nan"
rich_duke_df["CONTRAINDICATION"] = "nan"
rich_duke_df["WARNING"] = "nan"
rich_duke_df["ADVERSE"] = "nan"
rich_duke_df["POSOLOGY"] = "nan"

In [29]:
rich_duke_df.head()

,TAXON,ACTIVITY,CNAME,FAMILY,GENUS,SPECIES,VERNAC,CLINICAL,TRADITIONAL,FOLK,CONTRAINDICATION,WARNING,ADVERSE,POSOLOGY
0,Abelmoschus esculentus,"Abortifacient,Antidote,Boil,Burn,Catarrh,Coffe...","nan,Huang Shu K'Uei,Bamia,Molondron,Quiabeiro,...",Malvaceae,Abelmoschus,esculentus,nan,nan,nan,nan,nan,nan,nan,nan
1,Abelmoschus manihot,"Antitussive,Boil,Cancer,Cancer(Stomach),Catarr...","Tororo-Aoi,nan,Kastuli",Malvaceae,Abelmoschus,manihot,nan,nan,nan,nan,nan,nan,nan,nan
2,Abelmoschus moschatus,"Aphrodisiac,Asthma,Bite(Snake),Boil,Cancer,Car...","Musk Mallow,nan,Moskus,Muskus,Mushk Dana,Amber...",Malvaceae,Abelmoschus,moschatus,nan,nan,nan,nan,nan,nan,nan,nan
3,Abies alba,"Bronchitis,Bruise,Calculus,Catarrh,Cough,Diure...","Abeto,Edeltanne,Beyaz Koknar,Silver Fir,Abeto ...",Pinaceae,Abies,alba,nan,nan,nan,nan,nan,nan,nan,nan
4,Abies balsamea,"Burn,Cancer,Cold,Cough,Heart,Masticatory,Sore,...","nan,Balsam,Canada,Balsam Fir,Fir",Pinaceae,Abies,balsamea,nan,nan,nan,nan,nan,nan,nan,nan


Concatenating the dictionary values to the correct dataframe column:

In [30]:
def store_dict_values(remedy: dict, key: str):
    name_filt = (rich_duke_df["TAXON"] == remedy["NAME"])
    if  "nan" in rich_duke_df.loc[name_filt,[key]].values:
        rich_duke_df.loc[name_filt,[key]] = remedy[key]
    else:
        rich_duke_df.loc[name_filt,[key]] = rich_duke_df.loc[name_filt,[key]] + remedy[key]

In [31]:
skip_keys = ["NAME", "DEF"]

for i, herb in enumerate(who_01_remedies):
    for key in herb.keys():
        if key in skip_keys:
            continue
        store_dict_values(herb, key)

To better gauge the success of the previous code, we filter the dataframe to target one of the herbs that we know was in the monograph:

In [34]:
filt_alliumsativum = (rich_duke_df["TAXON"] == "Allium sativum")
rich_duke_df.loc[filt_alliumsativum]

,TAXON,ACTIVITY,CNAME,FAMILY,GENUS,SPECIES,VERNAC,CLINICAL,TRADITIONAL,FOLK,CONTRAINDICATION,WARNING,ADVERSE,POSOLOGY
473,Allium sativum,"Ache(Ear),Ache(Stomach),Alopecia,Antidote,Anti...","nan,Thum,Suan,Hsiao Suan,Sarimsak,Ail,Sir,Ajo,...",Liliaceae,Allium,sativum,"It is most commonly known as “garlic”,Ail,ail ...",As an adjuvant to dietetic management in the t...,The treatment of respiratory and urinary tract...,"As an aphrodisiac, antipyretic, diuretic, emme...",Bulbus Allii Sativi is contraindicated in pati...,Consumption of large amounts of garlic may inc...,Bulbus Allii Sativi has been reported to evoke...,"Unless otherwise prescribed, average daily dos..."


As we can see, this code successfully added the dictionary values to the dataframe.